In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D, Input,MaxPool2D, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam, SGD,Nadam,AdamW,RMSprop


import matplotlib.pyplot as plt
import numpy as np

from sklearn.metrics import roc_curve, auc, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np

In [2]:
model = tf.keras.models.load_model('ship_non_ship_cnn_model.keras')


for layer in model.layers:
    layer.trainable = False

# ---- Unfreeze only the last Conv2D and GlobalAveragePooling2D layers ----
# Find last Conv2D
last_conv2d = None
for layer in reversed(model.layers):
    if isinstance(layer, tf.keras.layers.Conv2D):
        last_conv2d = layer
        break

# Find GlobalAveragePooling2D
gap_layer = None
for layer in model.layers:
    if isinstance(layer, tf.keras.layers.GlobalAveragePooling2D):
        gap_layer = layer
        break

if last_conv2d is not None:
    last_conv2d.trainable = True
if gap_layer is not None:
    gap_layer.trainable = True

# ---- Recompile ----
model.compile(optimizer=Adam(learning_rate=5.38e-3), loss='binary_crossentropy', metrics=['accuracy'])


In [3]:
airplane_class = 0
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

def prepare_binary_dataset(x, y, target_class):
    target_indices = np.where(y == target_class)[0]
    other_indices = np.where(y != target_class)[0]
    np.random.shuffle(other_indices)
    other_indices = other_indices[:len(target_indices)]

    selected_indices = np.concatenate([target_indices, other_indices])
    np.random.shuffle(selected_indices)
    x_selected = x[selected_indices].astype('float32') / 255.0
    y_selected = np.where(y[selected_indices] == target_class, 1, 0)
    return x_selected, y_selected

x_train_airplane, y_train_airplane = prepare_binary_dataset(x_train, y_train, airplane_class)
x_test_airplane, y_test_airplane = prepare_binary_dataset(x_test, y_test, airplane_class)

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [4]:
def apply_cutout(image, size=8, n_holes=1):
    h, w = image.shape[0], image.shape[1]
    for n in range(n_holes):
        # Random position of cutout
        y = np.random.randint(h)
        x = np.random.randint(w)

        # Ensure cutout stays within image bounds
        y1 = np.clip(y - size // 2, 0, h)
        y2 = np.clip(y + size // 2, 0, h)
        x1 = np.clip(x - size // 2, 0, w)
        x2 = np.clip(x + size // 2, 0, w)

        # Set the cutout region to zero
        image[y1:y2, x1:x2, :] = 0
    return image
def cutout_preprocess(img):
    return apply_cutout(img)

# Data augmentation for airplane
datagen_airplane = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.12,
    shear_range=0.15,
    horizontal_flip=True,
    preprocessing_function=apply_cutout
)
datagen_airplane.fit(x_train_airplane)

callbacks_airplane = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)
]

print("Fine-tuning model on airplane and non-airplane images...")
history_airplane = model.fit(
    datagen_airplane.flow(x_train_airplane, y_train_airplane, batch_size=8),
    epochs=20,
    validation_data=(x_test_airplane, y_test_airplane),
    callbacks=callbacks_airplane,
    steps_per_epoch=len(x_train_airplane) // 8,
    verbose=1
)

model.save('airplane_non_airplane_cnn_model.keras')

Fine-tuning model on airplane and non-airplane images...
Epoch 1/20


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1250/1250 ━━━━━━━━━━━━━━━━━━━━ 54s 40ms/step - accuracy: 0.7175 - loss: 0.5795 - val_accuracy: 0.7790 - val_loss: 0.4880 - learning_rate: 0.0054
Epoch 2/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 83s 41ms/step - accuracy: 0.7524 - loss: 0.5314 - val_accuracy: 0.7625 - val_loss: 0.4900 - learning_rate: 0.0054
Epoch 3/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 77s 37ms/step - accuracy: 0.7455 - loss: 0.5363 - val_accuracy: 0.7715 - val_loss: 0.4930 - learning_rate: 0.0054
Epoch 4/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 89s 42ms/step - accuracy: 0.7456 - loss: 0.5337 - val_accuracy: 0.7800 - val_loss: 0.4811 - learning_rate: 0.0054
Epoch 5/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 75s 37ms/step - accuracy: 0.7524 - loss: 0.5283 - val_accuracy: 0.7705 - val_loss: 0.4903 - learning_rate: 0.0054
Epoch 6/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 48s 39ms/step - accuracy: 0.7444 - loss: 0.5434 - val_accuracy: 0.7880 - val_loss: 0.4664 - learning_rate: 0.0054
Epoch 7/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 81s 38ms/step - accuracy: 0.7560 